In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist  # Example dataset for simplicity, replace with 3D dataset

```Explanation of the code:
We import the necessary libraries for building, training, and visualizing our 3D GAN. TensorFlow is used to build the models, and matplotlib is used for visualization. We use the mnist dataset as a placeholder (replace with your own 3D data).```

In [2]:
def build_generator(latent_dim):
    model = tf.keras.Sequential()

    model.add(layers.Dense(4*4*4*512, activation="relu", input_dim=latent_dim))
    model.add(layers.Reshape((4, 4, 4, 512)))
    model.add(layers.Conv3DTranspose(256, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Conv3DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Conv3DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Conv3DTranspose(1, kernel_size=4, strides=2, padding="same", activation="sigmoid"))

    return model


```The Generator model is defined here. It takes a latent vector as input and outputs a 3D voxel grid. The model uses Dense layers and 3D convolutional transpose layers to upsample the latent vector into a 3D voxel grid.```

In [3]:
# def build_discriminator(input_shape):
#     model = tf.keras.Sequential()

#     model.add(layers.Conv3D(64, kernel_size=4, strides=2, padding="same", input_shape=input_shape))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(lers.Conv3D(128, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.Conv3D(256, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.Conv3D(512, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.Flatten())
#     model.add(layers.Dense(1, activation="sigmoid"))

#     return model
# #

In [4]:
def build_discriminator(input_shape):
    model = tf.keras.Sequential()

    model.add(layers.Conv3D(64, kernel_size=4, strides=2, padding="same", input_shape=input_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv3D(128, kernel_size=4, strides=2, padding="same")) # Changed 'lers' to 'layers'
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv3D(256, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv3D(512, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))

    return model

```The Discriminator model is defined here. It takes a 3D voxel grid as input and outputs a scalar value (0 for fake, 1 for real). The model uses 3D convolution layers followed by a Dense layer to produce a final output.```

In [5]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model


```The combined GAN model is defined here. The generator is connected to the discriminator to create a complete model. The discriminator’s weights are frozen during the generator's training to ensure that only the generator is updated.```

In [6]:
# Optimizers
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Loss Function
binary_crossentropy = tf.keras.losses.BinaryCrossentropy()

# Build and Compile the Discriminator
discriminator = build_discriminator((64, 64, 64, 1))
discriminator.compile(loss=binary_crossentropy, optimizer=optimizer, metrics=["accuracy"])

# Build the Generator
latent_dim = 100
generator = build_generator(latent_dim)

# Build the GAN (generator + discriminator)
gan = build_gan(generator, discriminator)
gan.compile(loss=binary_crossentropy, optimizer=optimizer)


c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


```We define the Adam optimizer for training and binary cross-entropy loss for both the generator and discriminator. The discriminator and GAN are compiled with these settings.```

In [7]:
def train_gan(generator, discriminator, gan, epochs, batch_size, latent_dim, dataset):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, dataset.shape[0], half_batch)
        real_voxels = dataset[idx]
        real_voxels = real_voxels.reshape((half_batch, 64, 64, 64, 1))

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_voxels = generator.predict(noise)

        # Labels for real and fake images
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Train on real and fake images
        d_loss_real = discriminator.train_on_batch(real_voxels, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_voxels, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator (through GAN model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print the progress
        if epoch % 100 == 0:
            print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")


```The training loop involves alternating between training the discriminator and the generator. The discriminator is trained on real and fake images, while the generator is trained through the combined GAN model to create realistic voxel grids.```

In [8]:
def plot_generated_voxel(voxels, epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    # Plot a batch of generated voxels
    plt.figure(figsize=figsize)
    for i in range(examples):
        ax = plt.subplot(dim[0], dim[1], i + 1)
        ax.imshow(voxels[i, :, :, 32, 0], cmap='gray')  # View the middle slice of the voxel grid
        ax.axis('off')
    plt.tight_layout()
    plt.savefig(f"generated_voxels_epoch_{epoch}.png")
    plt.close()


```The visualization function generates and saves images of the middle slices of the 3D voxel grids produced by the generator. This helps visualize the generated 3D data.```

In [13]:
# For the sake of demonstration, we'll use a 2D dataset (MNIST)
# Load dataset and preprocess it into 64x64x64 voxel grids
(X_train, _), (_, _) = mnist.load_data()
X_train = X_train.astype(np.float32) / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension
X_train = np.repeat(X_train, 64, axis=1)  # Dummy 3D data, expand to (64, 64, 64)
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension for consistency


MemoryError: Unable to allocate 11.2 GiB for an array with shape (60000, 1792, 28, 1) and data type float32

```This section loads and preprocesses the dataset. Here, we use the MNIST dataset as a placeholder, but for real use cases, you should replace this with actual 3D voxel datasets.```

In [12]:
X_train = X_train.reshape((-1, 64, 64, 64, 1))
epochs = 10000
batch_size = 64

train_gan(generator, discriminator, gan, epochs, batch_size, latent_dim, X_train)


ValueError: cannot reshape array of size 3010560000 into shape (64,64,64,1)

```This starts the training process, where the generator and discriminator are trained for the specified number of epochs and batch size.```